In [1]:
import colorsys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from timeit import default_timer as timer

import keras
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
from keras.utils import multi_gpu_model
gpu_num=1

#- Added
import cv2
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('../../movie2/media.io_IMG_5088.mp4')
camera_scale = 1.
#-

import csv
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# devide into car, walker, others
model = load_model('history/use_res_12.h5')

# color range
lower_red = [[140,50,50],[0,50,50]]
upper_red = [[180,255,255],[10,255,255]]

lower_green = [35, 50, 50]
upper_green = [95, 255, 255]

# setting the threshold
threshold = 90

In [3]:
class YOLO(object):
    def __init__(self):
        self.model_path = '../../model_data/yolo.h5' # model path or trained weights path
        self.anchors_path = '../../model_data/yolo_anchors.txt'
        self.classes_path = '../../model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.45
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416) # fixed size or (None, None), hw
        self.boxes, self.scores, self.classes = self.generate()
        self.count = 0
        self.num = 0
        self.zeroarray = np.zeros(25)

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        font = ImageFont.truetype(font='../../font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            if predicted_class == 'traffic light':
                label = '{} {:.2f}'.format(predicted_class, score)
                draw = ImageDraw.Draw(image)
                label_size = draw.textsize(label, font)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
                cropped_img = image.crop((left, top, right, bottom))
                file_name = f"../../img_data/rectangle/{self.count}.jpg"
                
                new_img = pil2cv(cropped_img)
                resize_img = np.array(cv2pil(cv2.resize(preprocess(new_img), dsize=(32, 32))))/255.0
                data = resize_img.reshape((1, 32, 32, 3))
#                 pred = model.predict(data)
#                 bestnum  = 0.0
#                 bestclass = 0
#                 for n in [0,1,2,3]:
#                     if bestnum < pred[0][n]:
#                         bestnum = pred[0][n]
#                         bestclass = n
#                 #color_prediction
#                 if (bestclass==0) or (bestclass==1):
                red = color_extraction(cropped_img, 'red', lower_red, upper_red)
                green = color_extraction(cropped_img, 'green', lower_green, upper_green)

                red_ratio = color_ratio(red, "red", threshold)
                green_ratio = color_ratio(green,  "green", threshold)
                string = color_predict(red_ratio, green_ratio)
                majority(string, self.zeroarray, self.num)
                self.num += 1
                self.count += 1

                if top - label_size[1] >= 0:
                    text_origin = np.array([left, top - label_size[1]])
                else:
                    text_origin = np.array([left, top + 1])

                # My kingdom for a good redistributable image drawing library.
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=self.colors[c])
                draw.rectangle(
                    [tuple(text_origin), tuple(text_origin + label_size)],
                    fill=self.colors[c])
                draw.text(text_origin, label, fill=(0, 0, 0), font=font)
                del draw

                end = timer()
        return image

    def close_session(self):
        self.sess.close()

In [4]:
def detect_img(yolo):
    while True:
        ret, image = cap.read()
        if cv2.waitKey(10) == 27:
            break
        h, w = image.shape[:2]
        rh = int(h * camera_scale)
        rw = int(w * camera_scale)
        image = cv2.resize(image, (rw, rh))
        image = image[:,:,(2,1,0)]
        image = Image.fromarray(image)
        r_image = yolo.detect_image(image)
        out_img = np.array(r_image)[:,:,(2,1,0)]
        cv2.imshow("YOLOv2", np.array(out_img))
#         cv2.waitKey(0)
    yolo.close_session()

def preprocess(img):
    h, w, c = img.shape
    longest_edge = max(h, w)
    top = 0
    bottom = 0
    left = 0
    right = 0
    if h < longest_edge:
        diff_h = longest_edge - h
        top = diff_h // 2
        bottom = diff_h - top
    elif w < longest_edge:
        diff_w = longest_edge - w
        left = diff_w // 2
        right = diff_w - left
    else:
        pass
    
    img = cv2.copyMakeBorder(img, top, bottom, left, right,
                             cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return img

def cv2pil(image):
    ''' OpenCV型 -> PIL型 '''
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image

def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

#Phase2:color extracting
#create color mask
def create_mask(color:str, lower_color:list, upper_color:list, img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    if color == 'red':
        lower_red1 = np.array(lower_color[0])
        upper_red1 = np.array(upper_color[0])
        lower_red2 = np.array(lower_color[1])
        upper_red2 = np.array(upper_color[1])
        red_mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        red_mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = red_mask1 + red_mask2
    else:
        lower_color = np.array(lower_color)
        upper_color = np.array(upper_color)
        mask = cv2.inRange(hsv, lower_color, upper_color)
    return mask

#create pics extracted the paricular color
def color_extraction(cropped_img:list, color:str, lower_color:list, upper_color:list):
    img = pil2cv(cropped_img)
    mask = create_mask(color, lower_color, upper_color, img)
    target = cv2.bitwise_and(img, img, mask=mask)
    return target

def color_ratio(color_pic:str, color:str, threshold:int):
    img = cv2.cvtColor(color_pic, cv2.COLOR_RGB2GRAY)
    ret, img_thresh = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    colorPixels = cv2.countNonZero(img_thresh)
    wholeSize = img.size
    color_ratio = colorPixels/wholeSize*100
    return color_ratio


def color_predict(red:list, green:list):
    if red > green:
        if red > 5:
            return('Red')
        else:
            return('NaN')
    elif green > red:            
        if green > 5:
            return('Green')
        else:
            return('NaN')
    else:
        return('NaN')


def majority(string:str, zerolist:list, num:int):
    if num < 25:
        fps = num
    else:
        fps = num - (num // 25) * 25
#     print(string)
    if 'Red' in string:
        zerolist[fps] = 0
    elif 'Green' in string:
        zerolist[fps] = 1
    else:
        zerolist[fps] = 'NaN'
    if fps == 24:
        nans = np.count_nonzero(np.isnan(zerolist))
        ones = np.count_nonzero(zerolist)-nans
        zeros = zerolist.shape[0]-ones-nans

        if (zeros > ones):
            print("'''Red!!!'''")
        elif (ones > zeros):
            print("'''Green!!!'''")
        else:
            print("'''idk!!!'''")

In [5]:
if __name__ == '__main__':
    detect_img(YOLO())

../../model_data/yolo.h5 model, anchors, and classes loaded.
'''Green!!!'''
'''Green!!!'''
'''Green!!!'''
'''Green!!!'''
'''Green!!!'''
'''Green!!!'''
'''Green!!!'''
'''Red!!!'''
'''idk!!!'''
'''Green!!!'''
'''Green!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Red!!!'''
'''Green!!!'''
'''Red!!!'''
'''Red!!!'''


AttributeError: 'NoneType' object has no attribute 'shape'